In [1]:
%load_ext autoreload
%autoreload 2
import os

import torch
from torch.utils.data import DataLoader
from datasets import TransformedRoboEireanData, RoboEireanDataWithEncoder
import utils
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from models import MultiClassJetNet
import torchvision.transforms as T
torch.manual_seed(2)

### RoboEirean Real Data

In [2]:
# # Default scalings for the default boxes determined by k-means clustering
default_box_scalings = torch.tensor(
    [
        [0.06549374, 0.12928654],
        [0.11965626, 0.26605093],
        [0.20708716, 0.38876095],
        [0.31018215, 0.47485098],
        [0.415882, 0.8048184],
        [0.7293086, 0.8216225],
    ]
)


classes = ["robot"]
encoder = utils.Encoder(default_box_scalings, ["robot"])
transformed_train_data = TransformedRoboEireanData(
    os.path.join("data", "transformed", "train"), encoder
)
transformed_val_data = TransformedRoboEireanData(
    os.path.join(
        "data",
        "transformed",
        "val",
    ),
    encoder,
)



### RoboEireann Augmented Data


In [2]:
# Default scalings for the default boxes determined by k-means clustering
default_box_scalings = torch.tensor(
    [
        [0.06549374, 0.12928654],
        [0.11965626, 0.26605093],
        [0.20708716, 0.38876095],
        [0.31018215, 0.47485098],
        [0.415882, 0.8048184],
        [0.7293086, 0.8216225],
    ]
)


classes = ["robot"]
encoder = utils.Encoder(default_box_scalings, ["robot"])

image_transforms = T.Compose(
            [
                T.Grayscale(),
                T.PILToTensor(),
                T.ConvertImageDtype(torch.float32),
                T.Resize((60, 80)),
            ]
        )
bounding_box_transforms = T.Compose([])

raw_train_data = RoboEireanDataWithEncoder(os.path.join("data", "raw", "train"),
                                           encoder,
                                           ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)

raw_val_data = RoboEireanDataWithEncoder(os.path.join("data", "raw", "val"),
                                           encoder,
                                           ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)
train_loader = DataLoader(
    raw_train_data, batch_size=32, shuffle=True, num_workers=1
)
val_loader = DataLoader(
    raw_val_data, batch_size=32, shuffle=False, num_workers=1
)

### COCO Synthetic Data


In [4]:
# default_box_scalings = torch.tensor(
#     [
#        [ 49.858948,  42.32408 ],
#        [ 79.69058 ,  96.98148 ],
#        [162.30188 , 125.641266],
#        [229.88889 , 248.09436 ],
#        [251.65385 , 434.61536 ],
#        [626.4     , 461.2     ]
#     ]
# )

# image_transforms = T.Compose(
#             [
#                 T.Grayscale(),
#                 T.PILToTensor(),
#                 T.ConvertImageDtype(torch.float32),
#                 T.Resize((60, 80)),
#             ]
#         )
# bounding_box_transforms = T.Compose([])

# raw_train_data = RoboEireanDataWithEncoder(os.path.join("data", "coco_ball_nao", "train"),
#                                            encoder,
#                                            ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)

# raw_val_data = RoboEireanDataWithEncoder(os.path.join("data", "coco_ball_nao", "val"),
#                                            encoder,
#                                            ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)
# train_loader = DataLoader(
#     raw_train_data, batch_size=32, shuffle=True, num_workers=1
# )
# val_loader = DataLoader(
#     raw_val_data, batch_size=32, shuffle=False, num_workers=1
# )

In [3]:
learning_rate = 2e-3
train_loader = DataLoader(
    raw_train_data, batch_size=32, shuffle=True, num_workers=0
)
val_loader = DataLoader(
    raw_val_data, batch_size=32, shuffle=True, num_workers=0
)

pl_model = MultiClassJetNet(len(classes), default_box_scalings.size(0), learning_rate)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=200, callbacks=[RichProgressBar()]
)
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy            │ MulticlassAccuracy │      0 │
│ 1 │ input_layer         │ NormConv2dReLU     │    146 │
│ 2 │ depth_wise_backbone │ Sequential         │  4.9 K │
│ 3 │ classifier          │ Sequential         │ 20.9 K │
│ 4 │ output_layer        │ Conv2d             │    900 │
└───┴─────────────────────┴────────────────────┴────────┘

Trainable params: 26.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 26.8 K                                                                                               
Total estimated model params size (MB): 0

/home/juell/Documents/GitHub/robot_detection/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/juell/Documents/GitHub/robot_detection/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connector
s/data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly
recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(

/home/juell/Documents/GitHub/robot_detection/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connector
s/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may 
be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/juell/Documents/GitHub/robot_detection/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.p
y:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), 
RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and 
Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), 
antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on
Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the 
models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(

selected predicted boxes is:  tensor([[-0.0227, -0.1087, -0.0343, -0.1258],
        [-0.0227, -0.1087, -0.0347, -0.1264],
        [ 0.1778,  0.0431,  0.0832,  0.1246],
        [ 0.0268,  0.1266,  0.0209, -0.1490],
        [-0.0214, -0.1083, -0.0355, -0.1257],
        [ 0.0259,  0.1257,  0.0215, -0.1473],
        [ 0.0257,  0.1255,  0.0220, -0.1482],
        [ 0.1666, -0.1400,  0.0086, -0.0573],
        [ 0.1860, -0.0225,  0.0085,  0.1224],
        [ 0.1658, -0.1394,  0.0097, -0.0580],
        [ 0.1860, -0.0226,  0.0089,  0.1222],
        [ 0.1659, -0.1392,  0.0099, -0.0578],
        [ 0.1858, -0.0223,  0.0082,  0.1223],
        [ 0.1657, -0.1395,  0.0090, -0.0572],
        [-0.0227, -0.1087, -0.0343, -0.1258],
        [ 0.0257,  0.1255,  0.0220, -0.1482],
        [ 0.1791,  0.0440,  0.0814,  0.1240],
        [ 0.0259,  0.1267,  0.0211, -0.1488],
        [ 0.1785,  0.0429,  0.0825,  0.1241],
        [ 0.0259,  0.1257,  0.0215, -0.1473],
        [-0.0220, -0.1080, -0.0347, -0.1259],
        [ 0.0268,  0.1266,  0.0208, -0.1490],
        [ 0.1660, -0.1400,  0.0086, -0.0571],
        [ 0.1660, -0.1400,  0.0086, -0.0571],
        [ 0.1661, -0.1400,  0.0086, -0.0572],
        [ 0.1661, -0.1401,  0.0087, -0.0571],
        [ 0.1661, -0.1400,  0.0084, -0.0573],
        [ 0.1661, -0.1400,  0.0085, -0.0573],
        [ 0.1661, -0.1400,  0.0085, -0.0574],
        [ 0.1784,  0.0437,  0.0827,  0.1239],
        [ 0.1781,  0.0436,  0.0829,  0.1240],
        [ 0.1859, -0.0218,  0.0079,  0.1220],
        [ 0.1796,  0.0437,  0.0814,  0.1234],
        [ 0.1788,  0.0439,  0.0814,  0.1242],
        [ 0.1862, -0.0222,  0.0082,  0.1220],
        [ 0.1783,  0.0431,  0.0827,  0.1251],
        [ 0.1855, -0.0212,  0.0080,  0.1231],
        [ 0.1855, -0.0225,  0.0088,  0.1217],
        [-0.0224, -0.1078, -0.0342, -0.1254],
        [ 0.0257,  0.1258,  0.0216, -0.1481],
        [ 0.1779,  0.0439,  0.0829,  0.1241],
        [ 0.0261,  0.1261,  0.0216, -0.1487],
        [-0.0228, -0.1087, -0.0350, -0.1265],
        [ 0.0265,  0.1262,  0.0213, -0.1488],
        [ 0.1791,  0.0440,  0.0814,  0.1240],
        [ 0.1858, -0.0227,  0.0086,  0.1220],
        [ 0.1796,  0.0437,  0.0814,  0.1234],
        [ 0.1848, -0.0220,  0.0092,  0.1219],
        [ 0.0253,  0.1261,  0.0217, -0.1483],
        [ 0.0271,  0.1267,  0.0209, -0.1489],
        [ 0.0271,  0.1257,  0.0219, -0.1483],
        [ 0.1789,  0.0435,  0.0817,  0.1244],
        [ 0.1858, -0.0225,  0.0086,  0.1222],
        [ 0.1789,  0.0435,  0.0818,  0.1243],
        [ 0.1859, -0.0225,  0.0085,  0.1222],
        [-0.0223, -0.1085, -0.0338, -0.1244],
        [-0.0229, -0.1090, -0.0337, -0.1250],
        [ 0.0262,  0.1263,  0.0211, -0.1489],
        [ 0.1788,  0.0439,  0.0814,  0.1242],
        [ 0.1854, -0.0224,  0.0094,  0.1223],
        [ 0.1789,  0.0435,  0.0817,  0.1244],
        [ 0.1858, -0.0225,  0.0086,  0.1222],
        [-0.0224, -0.1087, -0.0337, -0.1251],
        [-0.0240, -0.1093, -0.0331, -0.1247],
        [ 0.0257,  0.1258,  0.0214, -0.1478],
        [-0.0227, -0.1089, -0.0345, -0.1258],
        [ 0.0250,  0.1258,  0.0221, -0.1482],
        [ 0.0253,  0.1261,  0.0217, -0.1483],
        [-0.0223, -0.1085, -0.0346, -0.1264],
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]])
        [ 0.0271,  0.1267,  0.0209, -0.1489]

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
pl_model.eval()
batch = next(iter(val_loader))


In [ ]:
with torch.no_grad():
    predictions_single_batch = pl_model(batch[0])

In [ ]:
trainer.validate(model=pl_model, dataloaders=train_loader)

In [ ]:
trainer.validate(model=pl_model, dataloaders=val_loader)

In [ ]:
# from visualize import draw_model_output, image_grid
# import utils


# grid_size = 1000
# image_list = []
# for i in range(grid_size * grid_size):
#     image, encoded_bounding_boxes, target_masks, encoded_target_classes = raw_train_data[i]
#     predicted_boxes, predicted_class_logits = pl_model(image.unsqueeze(0))
#     predicted_classes = utils.calculate_predicted_classes(predicted_class_logits).squeeze()
#     print(predicted_classes)
#     decoded_boxes = encoder.decode_model_output(predicted_boxes, predicted_classes)
#     image_list.append(
#         draw_model_output(
#             image,
#             decoded_boxes,
#             predicted_classes,
#             torch.tensor([0, 1]),
#         )
#     )
# image_grid(image_list, grid_size, grid_size)
